In [1]:
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor


sam2_checkpoint = "/home/baraa.abdelsamad/SAM2-ChannelBased/ckpts/sam2_hiera_small.pt" # path to model weight (pre model loaded from: https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt)
model_cfg = "sam2_hiera_s.yaml" #  model config
sam2_model = build_sam2(model_cfg, sam2_checkpoint, device="cuda") # load model
predictor = SAM2ImagePredictor(sam2_model)
# do lora here 


/raid/home/baraa.abdelsamad/SAM2-ChannelBased/fine-tune-train_segment_anything_2_in_60_lines_of_code-main/sam2/modeling/sam/transformer.py:22: UserWarning: Flash Attention is disabled as it requires a GPU with Ampere (8.0) CUDA capability.
  OLD_GPU, USE_FLASH_ATTN, MATH_KERNEL_ON = get_sdpa_settings()


In [2]:
sam2_model

SAM2Base(
  (image_encoder): ImageEncoder(
    (trunk): Hiera(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
      )
      (blocks): ModuleList(
        (0): MultiScaleBlock(
          (norm1): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (attn): MultiScaleAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (proj): Linear(in_features=96, out_features=96, bias=True)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): MLP(
            (layers): ModuleList(
              (0): Linear(in_features=96, out_features=384, bias=True)
              (1): Linear(in_features=384, out_features=96, bias=True)
            )
            (act): GELU(approximate='none')
          )
        )
        (1): MultiScaleBlock(
          (norm1): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
      

In [4]:
# Iterate over the model's named parameters and print the trainable ones
count = 0
for param in sam2_model.parameters():
    if param.requires_grad:
        # print(f"Trainable parameter: {name}, Shape: {param.shape}")
        count+=1
print(f"trainable params before lora: {count}")


trainable params before lora: 515


In [5]:
from peft import LoraConfig, get_peft_model

# Define the LoRA configuration
config = LoraConfig(
    r=8, 
    lora_alpha=8,   
    target_modules=[
        # sam_mask_decoder layers
        "sam_mask_decoder.transformer.layers.0.self_attn.q_proj",
        "sam_mask_decoder.transformer.layers.0.self_attn.k_proj",
        "sam_mask_decoder.transformer.layers.0.self_attn.v_proj",
        "sam_mask_decoder.transformer.layers.0.self_attn.out_proj",
        "sam_mask_decoder.transformer.layers.0.cross_attn_token_to_image.q_proj",
        "sam_mask_decoder.transformer.layers.0.cross_attn_token_to_image.k_proj",
        "sam_mask_decoder.transformer.layers.0.cross_attn_token_to_image.v_proj",
        "sam_mask_decoder.transformer.layers.0.cross_attn_token_to_image.out_proj",
        "sam_mask_decoder.transformer.layers.0.cross_attn_image_to_token.q_proj",
        "sam_mask_decoder.transformer.layers.0.cross_attn_image_to_token.k_proj",
        "sam_mask_decoder.transformer.layers.0.cross_attn_image_to_token.v_proj",
        "sam_mask_decoder.transformer.layers.0.cross_attn_image_to_token.out_proj",
        
        # memory_attention layers
        "memory_attention.layers.0.self_attn.q_proj",
        "memory_attention.layers.0.self_attn.k_proj",
        "memory_attention.layers.0.self_attn.v_proj",
        "memory_attention.layers.0.self_attn.out_proj",
        "memory_attention.layers.0.cross_attn_image.q_proj",
        "memory_attention.layers.0.cross_attn_image.k_proj",
        "memory_attention.layers.0.cross_attn_image.v_proj",
        "memory_attention.layers.0.cross_attn_image.out_proj"
    ],
    lora_dropout=0.0,
    bias="none",
)

# Apply the LoRA configuration to the model
model = get_peft_model(sam2_model, config)


# # Freeze the entire model
# for param in model.parameters():
#     param.requires_grad = False

# # Unfreeze only the LoRA layers
# for name, param in model.named_parameters():
#     if 'lora' in name:
#         param.requires_grad = True

/home/baraa.abdelsamad/anaconda3/envs/SAM2-channels/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model

PeftModel(
  (base_model): LoraModel(
    (model): SAM2Base(
      (image_encoder): ImageEncoder(
        (trunk): Hiera(
          (patch_embed): PatchEmbed(
            (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          )
          (blocks): ModuleList(
            (0): MultiScaleBlock(
              (norm1): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
              (attn): MultiScaleAttention(
                (qkv): Linear(in_features=96, out_features=288, bias=True)
                (proj): Linear(in_features=96, out_features=96, bias=True)
              )
              (drop_path): Identity()
              (norm2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
              (mlp): MLP(
                (layers): ModuleList(
                  (0): Linear(in_features=96, out_features=384, bias=True)
                  (1): Linear(in_features=384, out_features=96, bias=True)
                )
                (act): GELU(approximate='none')

In [7]:
# Iterate over the model's named parameters and print the trainable ones
count = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Trainable parameter: {name}, Shape: {param.shape}")
        count+=1
print(f"trainable params after lora: {count}")


Trainable parameter: base_model.model.memory_attention.layers.0.self_attn.q_proj.lora_A.default.weight, Shape: torch.Size([8, 256])
Trainable parameter: base_model.model.memory_attention.layers.0.self_attn.q_proj.lora_B.default.weight, Shape: torch.Size([256, 8])
Trainable parameter: base_model.model.memory_attention.layers.0.self_attn.k_proj.lora_A.default.weight, Shape: torch.Size([8, 256])
Trainable parameter: base_model.model.memory_attention.layers.0.self_attn.k_proj.lora_B.default.weight, Shape: torch.Size([256, 8])
Trainable parameter: base_model.model.memory_attention.layers.0.self_attn.v_proj.lora_A.default.weight, Shape: torch.Size([8, 256])
Trainable parameter: base_model.model.memory_attention.layers.0.self_attn.v_proj.lora_B.default.weight, Shape: torch.Size([256, 8])
Trainable parameter: base_model.model.memory_attention.layers.0.self_attn.out_proj.lora_A.default.weight, Shape: torch.Size([8, 256])
Trainable parameter: base_model.model.memory_attention.layers.0.self_attn.

In [11]:
import yaml
from collections import OrderedDict

def layer_to_yaml(layer, prefix=""):
    """Recursively convert model layers into a YAML-compatible dictionary."""
    yaml_dict = OrderedDict()

    for name, sublayer in layer.named_children():
        full_name = f"{prefix}{name}"

        if len(list(sublayer.named_children())) > 0:
            # Recursively process sublayers
            yaml_dict[name] = layer_to_yaml(sublayer, prefix="")
        else:
            # Leaf layer, consider special cases for LoRA layers
            if isinstance(sublayer, torch.nn.Linear):
                yaml_dict[name] = {
                    "_target_": "torch.nn.Linear",
                    "in_features": sublayer.in_features,
                    "out_features": sublayer.out_features,
                    "bias": sublayer.bias is not None
                }
            elif isinstance(sublayer, torch.nn.Conv2d):
                yaml_dict[name] = {
                    "_target_": "torch.nn.Conv2d",
                    "in_channels": sublayer.in_channels,
                    "out_channels": sublayer.out_channels,
                    "kernel_size": sublayer.kernel_size,
                    "stride": sublayer.stride,
                    "padding": sublayer.padding
                }
            elif isinstance(sublayer, lora.Linear):
                yaml_dict[name] = {
                    "_target_": "lora.Linear",
                    "in_features": sublayer.base_layer.in_features,
                    "out_features": sublayer.base_layer.out_features,
                    "bias": sublayer.base_layer.bias is not None,
                    "lora_A": sublayer.lora_A["default"].weight.shape,
                    "lora_B": sublayer.lora_B["default"].weight.shape,
                }
            else:
                yaml_dict[name] = {"_target_": str(type(sublayer)).split("'")[1]}

    return yaml_dict

def model_to_yaml(model):
    """Convert the entire model into a YAML structure."""
    yaml_structure = layer_to_yaml(model.base_model.model)
    return yaml_structure

def save_yaml(yaml_structure, filename):
    """Save the YAML structure to a file."""
    with open(filename, 'w') as file:
        yaml.dump(yaml_structure, file, default_flow_style=False, sort_keys=False)

# Assuming `model` is your LoRA model
yaml_structure = model_to_yaml(model)
save_yaml(yaml_structure, "model_config.yaml")


In [ ]:


class Adapter(nn.Module):
    def __init__(self, input_dim, adapter_dim):
        super(Adapter, self).__init__()
        self.fc = nn.Linear(input_dim, adapter_dim)
        self.norm = nn.LayerNorm(adapter_dim)
    
    def forward(self, x):
        x = self.fc(x)
        x = self.norm(x)
        return x

class update_model:
    def __init__(self, model) -> None:
        self.model = model
        self.mlp_head = copy.deepcopy(model.mlp_head)

    def add_adapter(self, model, adapter_dim=768):
        # Assuming that 'v' is the attribute name of the transformer in model
        transformer = getattr(model, 'v', None)
        if transformer is None:
            raise ValueError("Model does not contain 'v' attribute. Please provide a model that contains the transformer.")
        
        # Add adapter after the transformer
        setattr(model, 'adapter', Adapter(input_dim=768, adapter_dim=adapter_dim))  # Assuming the input dimension is 768
        
        # Modify the 'mlp_head' to accept the output dimension of the adapter
        setattr(model, 'mlp_head', nn.Sequential(
            nn.LayerNorm(adapter_dim),  # Apply LayerNorm before the adapter output
            nn.Linear(adapter_dim, 3)   # Output dimension 3, change if needed
        ))
        
        return model

    def freeze_and_add_adapter_layers(self):
        for name, param in self.model.v.named_parameters():
            if "adapter" not in name:
                param.requires_grad = False
        model_with_adapter = self.add_adapter(self.model)
        return model_with_adapter

def count_trainable_params(model):
    count = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, param.shape)
            count+=1
    return count

# Dataset read

In [23]:

import pandas as pd 
import os 
# Path to the CSV file and dataset directory
csv_file = "/home/baraa.abdelsamad/SAM2-ChannelBased/data/RA_train.csv"  # Replace with your CSV file path
data_dir = "/mnt/rcl-server/workspace/baraa/echo-segmentation-2-resized/"  # Path to dataset (LabPics 1)

# Read the CSV file
df = pd.read_csv(csv_file)

# List to store image and annotation paths
data = []

for index, row in df.iterrows():
    filename = row['file_name']  # Access the filename from the 'filename' column in the CSV
    print(os.path.join(os.path.join(data_dir, "images/"), filename+ ".png"))
    image_path = os.path.join(os.path.join(data_dir, "images/"), filename+ ".png")
    annotation_path = os.path.join(os.path.join(data_dir, "/masks/"), filename + ".png")
    
    # Add to the data list
    data.append({"image": image_path, "annotation": annotation_path})

/mnt/rcl-server/workspace/baraa/echo-segmentation-2-resized/images/2.25.303205808567141670970926639986321802310_2.25.23936708830284570006874049250312454990_2__RA_Vol_MOD_A4C_calc.png
/mnt/rcl-server/workspace/baraa/echo-segmentation-2-resized/images/2.25.59725375199207978909088863690773612851_2.25.44982182398041843845843462117706089313_2__RA_Vol_MOD_A4C_calc.png
/mnt/rcl-server/workspace/baraa/echo-segmentation-2-resized/images/2.25.311516365306464807403881746301462266325_2.25.169863032067677783560162238221456720886_2__RA_Vol_MOD_A4C_calc.png
/mnt/rcl-server/workspace/baraa/echo-segmentation-2-resized/images/2.25.332574525137427117254190066974520945344_2.25.142988026657904742104364910126429167004_2__RA_Vol_MOD_A4C_calc.png
/mnt/rcl-server/workspace/baraa/echo-segmentation-2-resized/images/2.25.42981473258215486791380363625819314136_2.25.172645484338545574666035088616154496710_2__RA_Vol_MOD_A4C_calc.png
/mnt/rcl-server/workspace/baraa/echo-segmentation-2-resized/images/2.25.305072487928

In [24]:

# Optionally, print the first few entries to verify
print(data[:5])

[{'image': '/mnt/rcl-server/workspace/baraa/echo-segmentation-2-resized/images/2.25.303205808567141670970926639986321802310_2.25.23936708830284570006874049250312454990_2__RA_Vol_MOD_A4C_calc.png', 'annotation': '/masks/2.25.303205808567141670970926639986321802310_2.25.23936708830284570006874049250312454990_2__RA_Vol_MOD_A4C_calc.png'}, {'image': '/mnt/rcl-server/workspace/baraa/echo-segmentation-2-resized/images/2.25.59725375199207978909088863690773612851_2.25.44982182398041843845843462117706089313_2__RA_Vol_MOD_A4C_calc.png', 'annotation': '/masks/2.25.59725375199207978909088863690773612851_2.25.44982182398041843845843462117706089313_2__RA_Vol_MOD_A4C_calc.png'}, {'image': '/mnt/rcl-server/workspace/baraa/echo-segmentation-2-resized/images/2.25.311516365306464807403881746301462266325_2.25.169863032067677783560162238221456720886_2__RA_Vol_MOD_A4C_calc.png', 'annotation': '/masks/2.25.311516365306464807403881746301462266325_2.25.169863032067677783560162238221456720886_2__RA_Vol_MOD_A4C_

In [25]:
import numpy as np
import cv2

def read_batch(data): # read random image and its annotaion from  the dataset (LabPics)

   #  select image

        ent  = data[np.random.randint(len(data))] # choose random entry
        print(ent['image'])
        Img = cv2.imread(ent["image"])  # read image
        # Convert the image and mask to numpy arrays
        image_np = np.array(Img)
        # If the image is 2D (grayscale), convert it to 3D by stacking
        if len(Img.shape) == 2: # use this in training 
            image_np = np.stack([Img] * 3, axis=-1)

        gt = cv2.imread(ent["annotation"]) # read annotation

   # resize image

        r = np.min([1024 / Img.shape[1], 1024 / Img.shape[0]]) # scalling factor
        Img = cv2.resize(Img, (int(Img.shape[1] * r), int(Img.shape[0] * r)))
        ann_map = cv2.resize(ann_map, (int(ann_map.shape[1] * r), int(ann_map.shape[0] * r)),interpolation=cv2.INTER_NEAREST)

   # merge vessels and materials annotations

        mat_map = ann_map[:,:,0] # material annotation map
        ves_map = ann_map[:,:,2] # vessel  annotaion map
        mat_map[mat_map==0] = ves_map[mat_map==0]*(mat_map.max()+1) # merge maps

   # Get binary masks and points

        inds = np.unique(mat_map)[1:] # load all indices
        points= []
        masks = []
        for ind in inds:
            mask=(mat_map == ind).astype(np.uint8) # make binary mask corresponding to index ind
            masks.append(mask)
            coords = np.argwhere(mask > 0) # get all coordinates in mask
            yx = np.array(coords[np.random.randint(len(coords))]) # choose random point/coordinate
            points.append([[yx[1], yx[0]]])
        return Img,np.array(masks),np.array(points), np.ones([len(masks),1])

image,mask,input_point, input_label = read_batch(data)

/mnt/rcl-server/workspace/baraa/echo-segmentation-2-resized/images/2.25.307273518699099444520456210042600901071_2.25.294111046415329599808260776880976786242_2__RA_Vol_MOD_A4C_calc.png


[ WARN:0@398.817] global loadsave.cpp:241 findDecoder imread_('/masks/2.25.307273518699099444520456210042600901071_2.25.294111046415329599808260776880976786242_2__RA_Vol_MOD_A4C_calc.png'): can't open/read file: check file path/integrity


UnboundLocalError: cannot access local variable 'ann_map' where it is not associated with a value

In [1]:
import torch 
ckpt_path = "/home/baraa.abdelsamad/SAM2-ChannelBased/model.torch"
ckpt = torch.load(ckpt_path, map_location="cpu")

/tmp/ipykernel_26061/2013769747.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location="cpu")


In [3]:
for key in ckpt.keys():
    print(key)

base_model.model.maskmem_tpos_enc
base_model.model.no_mem_embed
base_model.model.no_mem_pos_enc
base_model.model.no_obj_ptr
base_model.model.image_encoder.trunk.pos_embed
base_model.model.image_encoder.trunk.pos_embed_window
base_model.model.image_encoder.trunk.patch_embed.proj.weight
base_model.model.image_encoder.trunk.patch_embed.proj.bias
base_model.model.image_encoder.trunk.blocks.0.norm1.weight
base_model.model.image_encoder.trunk.blocks.0.norm1.bias
base_model.model.image_encoder.trunk.blocks.0.attn.qkv.weight
base_model.model.image_encoder.trunk.blocks.0.attn.qkv.bias
base_model.model.image_encoder.trunk.blocks.0.attn.proj.weight
base_model.model.image_encoder.trunk.blocks.0.attn.proj.bias
base_model.model.image_encoder.trunk.blocks.0.norm2.weight
base_model.model.image_encoder.trunk.blocks.0.norm2.bias
base_model.model.image_encoder.trunk.blocks.0.mlp.layers.0.weight
base_model.model.image_encoder.trunk.blocks.0.mlp.layers.0.bias
base_model.model.image_encoder.trunk.blocks.0.m

In [4]:
import logging
import torch

def load_checkpoint(model, ckpt_path):
    if ckpt_path is not None:
        # Load the entire checkpoint
        checkpoint = torch.load(ckpt_path, map_location="cpu")

        # Extract the 'base_model' key (assuming it's a nested dictionary)
        sd = checkpoint.get("base_model", checkpoint)

        # Initialize a new state dictionary
        new_state_dict = {}

        # Iterate through the state dict and strip 'base_model' prefix and 'base_layer' suffix
        for key, value in sd.items():
            # Remove the 'base_model.model.' prefix if present
            if key.startswith("base_model.model."):
                new_key = key[len("base_model.model."):]
            else:
                new_key = key

            # Remove the '.base_layer' suffix if present
            if new_key.endswith(".base_layer.weight"):
                new_key = new_key.replace(".base_layer.weight", ".weight")
            elif new_key.endswith(".base_layer.bias"):
                new_key = new_key.replace(".base_layer.bias", ".bias")

            # Add the cleaned key-value pair to the new state dictionary
            new_state_dict[new_key] = value

        # Load the modified state dictionary into the model
        missing_keys, unexpected_keys = model.load_state_dict(new_state_dict, strict=False)

        # Handle missing or unexpected keys
        if missing_keys:
            logging.error(f"Missing keys: {missing_keys}")
            raise RuntimeError(f"Failed to load checkpoint: missing keys: {missing_keys}")
        if unexpected_keys:
            logging.error(f"Unexpected keys: {unexpected_keys}")
            raise RuntimeError(f"Failed to load checkpoint: unexpected keys: {unexpected_keys}")

        logging.info("Loaded checkpoint successfully")
